In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm

In [2]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def CSHam_ex(N, B, Ak, GS, Filling):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # now we add the extra gap filler
    #GS_s = scipy.sparse.csr_matrix(GS)
    #fill_Op = Filling * scipy.sparse.kron(GS_s, scipy.sparse.spmatrix.getH(GS_s))
    fill_Op = Filling * np.outer(GS, np.conj(GS))
    operators.append(fill_Op)
    sites.append(np.arange(0,N).tolist())
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr

In [3]:
# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.05)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.5) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.variational.MCState(self.sampler, self.machine, n_samples=1000, n_discard=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output):
        self.vs.init_parameters(nk.nn.initializers.normal(stddev=0.25))
        gs = nk.VMC(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, sr=self.sr)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=1000)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, runTime
    
# Combines all steps into a function to run on the cluster
def runDescentCS_ex(N,B,Ak,alpha,GS,Filling):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam_ex(N,B,Ak,GS,Filling)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/Logs"+str(N)) #where _call_ will be invoked
    return eng, state, runTime

# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/Logs"+str(N)) #where _call_ will be invoked
    return eng, state, runTime

In [7]:
Ak = []
for i in range(4):
        # Constant A
    Ak_i = 1
        # Variable A
        #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)

ha, hi = CSHam(5,0.95,Ak)
ham = ha.to_dense()

e,v  = exactDiagonalization_full(ha)
print('exact ground state is',e[0],v[:][0])
print('exact 1st excited state is', e[1], v[:][1])


exact ground state is -1.8313364600868471 [ 0.00000000e+00+0.j  1.67831314e-01+0.j  1.67831314e-01+0.j
  5.55111512e-17+0.j  1.67831314e-01+0.j  1.26800801e-17+0.j
  1.26800801e-17+0.j -6.16297582e-33+0.j  1.67831314e-01+0.j
  1.26800801e-17+0.j  1.26800801e-17+0.j -3.35801691e-34+0.j
  8.59666449e-18+0.j -3.35801691e-34+0.j -3.35457541e-34+0.j
  0.00000000e+00+0.j -9.41982272e-01+0.j -3.86622615e-17+0.j
 -3.86622615e-17+0.j  7.18106801e-34+0.j -1.98268007e-17+0.j
  7.18106801e-34+0.j  7.16953506e-34+0.j  0.00000000e+00+0.j
 -1.98268007e-17+0.j  2.47542833e-34+0.j  2.47451788e-34+0.j
  0.00000000e+00+0.j  2.47451788e-34+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j]
exact 1st excited state is -1.673244532749028 [ 0.00000000e+00+0.j  1.36438465e-17+0.j  0.00000000e+00+0.j
 -2.02196385e-01+0.j  2.77555756e-17+0.j -2.02196385e-01+0.j
 -2.02196385e-01+0.j  1.11022302e-16+0.j  9.71445147e-17+0.j
 -2.02196385e-01+0.j -2.02196385e-01+0.j  1.72600518e-17+0.j
 -2.02196385e-01

In [4]:

N = 5
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
#A = N/2
#N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization(ha)
#Ground state energy
edEng = e[0]
# Ground state
edState = v[0]

#Lists for Histogram Data
numRuns = 10
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []
GS = []

# Get errors for each run in histogram
for i in range(len(hisIt)):
    engTemp, stateTemp, runTimeTemp = runDescentCS(N,B,Ak,alpha)
    #engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    GS.append(np.asmatrix(stateTemp))
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)
print(GS)


100%|██████████| 1000/1000 [00:18<00:00, 52.63it/s, Energy=-1.83012+0.00010j ± 0.00035 [σ²=0.00013, R̂=1.0245]]


Eng error  [0.0012153903728830961]
State error  [0.016696514439110577]
[matrix([[ 1.94341691e-03+2.21286365e-02j,
          4.78351249e-03+1.65004816e-01j,
          4.72504306e-03+1.65742550e-01j,
          9.83003197e-04+2.43442472e-02j,
          5.05433047e-03+1.65338562e-01j,
         -4.24850305e-03+3.16964521e-02j,
         -9.90573532e-04+2.74018019e-02j,
         -1.88394833e-03+3.44828966e-03j,
          4.79202341e-03+1.66154054e-01j,
         -1.26503909e-03+2.66406244e-02j,
         -1.93685510e-03+2.42316635e-02j,
         -2.46106740e-03+3.03821116e-03j,
         -1.02394765e-03+2.52643718e-02j,
         -2.23567352e-03+1.28157831e-03j,
         -3.31009249e-03+4.03466071e-03j,
         -1.21910402e-04+3.84825963e-03j,
         -2.80866069e-02-9.34228791e-01j,
          5.15505622e-03-5.34701715e-02j,
          2.10329133e-03-5.30710961e-02j,
          2.50920517e-05-2.66737791e-03j,
          6.68714793e-03-5.56411664e-02j,
          1.01988559e-03-3.39172866e-03j,
    

In [16]:
GS_s= scipy.sparse.csr_matrix(GS[0])
print(scipy.sparse.kron(GS_s, scipy.sparse.spmatrix.getH(GS_s)))

  (0, 0)	(0.0004934534224622913+0j)
  (1, 0)	(0.003660627949125049-0.00021482053995005066j)
  (2, 0)	(0.0036768393757319108-0.00021754811421710878j)
  (3, 0)	(0.0005406153817081615-2.5558501154936264e-05j)
  (4, 0)	(0.003668539608085879-0.00020947631527917066j)
  (5, 0)	(0.0006931426547799568-0.0001556130006453451j)
  (6, 0)	(0.0006044394161606551-7.517316672087182e-05j)
  (7, 0)	(7.264465138074602e-05-4.8390672192527195e-05j)
  (8, 0)	(0.0036860755520917816-0.00021686565280788455j)
  (9, 0)	(0.0005870621945206363-7.976743009712923e-05j)
  (10, 0)	(0.0005324495574125031-8.995218719935569e-05j)
  (11, 0)	(6.244859034661004e-05-6.036457686116304e-05j)
  (12, 0)	(0.0005570761425878728-7.17577726605609e-05j)
  (13, 0)	(2.401473478877982e-05-5.196304754669481e-05j)
  (14, 0)	(8.284865056234174e-05-8.108886138428041e-05j)
  (15, 0)	(8.49198157293861e-05-1.017648378975377e-05j)
  (16, 0)	(-0.020727793313793413+0.0011940777142962484j)
  (17, 0)	(-0.0011732035661371624+0.00021798920062011102j)


In [29]:

N = 5
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
#A = N/2
#N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
# Define hamiltonian and hilbert space
ha, hi = CSHam_ex(N,B,Ak,GS[0],0.5)

#Exact Diagonalization
e, v = exactDiagonalization(ha)
#Ground state energy
edEng = e[0]
# Ground state
edState = v[0]

#Lists for Histogram Data
numRuns = 1
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []
GS_ex = []

# Get errors for each run in histogram
for i in range(len(hisIt)):
    engTemp, stateTemp, runTimeTemp = runDescentCS_ex(N,B,Ak,alpha,GS[0],0.5)
    #engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    GS_ex.append(np.asmatrix(stateTemp))
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)
print(GS_ex)


100%|██████████| 1000/1000 [00:29<00:00, 34.20it/s, Energy=-1.67621-0.00003j ± 0.00073 [σ²=0.00054, R̂=1.0035]]


Eng error  [0.0006897141919872674]
State error  [0.01365866007827643]
[matrix([[ 0.00415183-9.90428564e-03j,  0.00043062+3.05500039e-02j,
         -0.00086696+3.06108283e-02j,  0.03174565-1.94246001e-01j,
         -0.00039337+3.35574975e-02j,  0.03178011-1.94138408e-01j,
          0.03140973-1.96104602e-01j, -0.02243456-2.49175953e-02j,
          0.00083157+2.99474077e-02j,  0.0318727 -1.94607483e-01j,
          0.0318594 -1.94263361e-01j, -0.02987617-2.49118547e-02j,
          0.03193516-1.93614616e-01j, -0.02842008-2.82035044e-02j,
         -0.02262515-2.59300388e-02j,  0.01027053-1.22556289e-02j,
          0.00317064-1.73011123e-01j, -0.06799183+4.17981799e-01j,
         -0.0693658 +4.18892925e-01j,  0.0300345 +2.29983775e-02j,
         -0.06645439+4.19969262e-01j,  0.01891263+2.93021539e-02j,
          0.01840208+2.26212368e-02j, -0.00300743-2.93179704e-04j,
         -0.06909194+4.17456721e-01j,  0.03047825+2.31294307e-02j,
          0.0329202 +2.10056696e-02j, -0.00788314+2.588909

In [30]:
ha, hi = CSHam(5,0.95,Ak)
ham = ha.to_dense()

e,v  = exactDiagonalization_full(ha)

def err_s(state, edState, N):
    #engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return  waveFunctionErr

print(err_s(np.array(v[:][1]),np.array(GS_ex),5))


0.04753422584669065


In [22]:
ha, hi = CSHam_ex(5,0.95,Ak,GS,-2)
ham = ha.to_dense()

e,v_ex  = exactDiagonalization_full(ha)
print('exact ground state is',e[0],v_ex[:][0])
print('exact 1st excited state is', e[1], v_ex[:][1])
print(err_s(v_ex[:][0],v[:][0],5))

exact ground state is -3.8282403764572925 [-8.37616922e-03+0.j         -1.65838047e-01-0.00963299j
 -1.66141875e-01-0.00968411j -2.28030026e-02-0.00231128j
 -1.65986324e-01-0.00953283j -2.62557042e-02-0.0054883j
 -2.43253852e-02-0.00351333j -2.69463189e-03-0.00159448j
 -1.66314975e-01-0.00967132j -2.37967493e-02-0.00375673j
 -2.25795782e-02-0.00367756j -2.35219816e-03-0.00178864j
 -2.33567003e-02-0.00375333j -1.52107273e-03-0.00165702j
 -2.87391194e-03-0.00245317j -2.48790445e-03-0.00045495j
  9.34058152e-01+0.0538627j   4.97819597e-02+0.00890272j
  4.91388954e-02+0.00638766j  2.33580463e-03+0.00065853j
  5.13933951e-02+0.01031554j  2.59028308e-03+0.00126766j
  2.49158724e-03+0.00148994j  4.16696679e-04+0.00013723j
  4.93292426e-02+0.00906813j  2.25815808e-03+0.00074441j
  2.05470222e-03+0.0013342j   3.33163914e-04+0.00045598j
  2.39794108e-03+0.00123655j  2.73970354e-04+0.0001246j
  1.23442633e-04+0.00045563j  2.48338730e-04+0.00055771j]
exact 1st excited state is -1.6755903763400501 